In [1]:
# hyperparameter tuning

# Import necessary libraries
import mlflow
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np
import os
import dagshub

In [2]:
reviews = pd.read_csv(r"C:\Users\work\Documents\Kaggle\train\amazon_reviews_train_clean.csv")  # read kaggle cleaned data
reviews.isna().sum()
reviews['text'].fillna("", inplace=True)
reviews.isna().sum()

C:\Users\work\AppData\Local\Temp\ipykernel_18980\1184438437.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  reviews['text'].fillna("", inplace=True)


label    0
text     0
dtype: int64

In [3]:
import dagshub
mlflow.set_tracking_uri('https://dagshub.com/kunalkaushik1982/amazon-product-reviews.mlflow')
dagshub.init(repo_owner='kunalkaushik1982', repo_name='amazon-product-reviews', mlflow=True)

# Set the experiment name
mlflow.set_experiment("LoR Hyperparameter Tuning")

Accessing as kunalkaushik1982

Initialized MLflow to track repo "kunalkaushik1982/amazon-product-reviews"

Repository kunalkaushik1982/amazon-product-reviews initialized!

2024/12/29 20:53:53 INFO mlflow.tracking.fluent: Experiment with name 'LoR Hyperparameter Tuning' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/b4d1363daa5f48d5a7171a6a9eb9d5bb', creation_time=1735485833873, experiment_id='3', last_update_time=1735485833873, lifecycle_stage='active', name='LoR Hyperparameter Tuning', tags={}>

In [4]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(reviews['text'])
y = reviews['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [11]:
param_grid = {
    'C': [0.1, 1, 10],
    'penalty': ['l1', 'l2'],
    'solver': ['liblinear']
}

In [12]:
# Start the parent run for hyperparameter tuning
with mlflow.start_run():

    # Perform grid search
    grid_search = GridSearchCV(LogisticRegression(), param_grid, cv=5, scoring='f1', n_jobs=-1)
    grid_search.fit(X_train, y_train)

    # Log each parameter combination as a child run
    for params, mean_score, std_score in zip(grid_search.cv_results_['params'], grid_search.cv_results_['mean_test_score'], grid_search.cv_results_['std_test_score']):
        with mlflow.start_run(run_name=f"LR with params: {params}", nested=True):
            model = LogisticRegression(**params)
            model.fit(X_train, y_train)
            
            # Model evaluation
            y_pred = model.predict(X_test)
            accuracy = accuracy_score(y_test, y_pred)
            precision = precision_score(y_test, y_pred)
            recall = recall_score(y_test, y_pred)
            f1 = f1_score(y_test, y_pred)
            
            # Log parameters and metrics
            mlflow.log_params(params)
            mlflow.log_metric("mean_cv_score", mean_score)
            mlflow.log_metric("std_cv_score", std_score)
            mlflow.log_metric("accuracy", accuracy)
            mlflow.log_metric("precision", precision)
            mlflow.log_metric("recall", recall)
            mlflow.log_metric("f1_score", f1)
            
            
            # Print the results for verification
            print(f"Mean CV Score: {mean_score}, Std CV Score: {std_score}")
            print(f"Accuracy: {accuracy}")
            print(f"Precision: {precision}")
            print(f"Recall: {recall}")
            print(f"F1 Score: {f1}")

    # Log the best run details in the parent run
    best_params = grid_search.best_params_
    best_score = grid_search.best_score_
    mlflow.log_params(best_params)
    mlflow.log_metric("best_f1_score", best_score)
    
    print(f"Best Params: {best_params}")
    print(f"Best F1 Score: {best_score}")

    # Save and log the notebook                
    notebook_path = "exp3_lor_bow_hp.ipynb"
    os.system(f"jupyter nbconvert --to notebook --execute --inplace {notebook_path}")
    mlflow.log_artifact(notebook_path)

    # Log model
    mlflow.sklearn.log_model(grid_search.best_estimator_, "model")

🏃 View run merciful-ant-991 at: https://dagshub.com/kunalkaushik1982/amazon-product-reviews.mlflow/#/experiments/3/runs/154e08df25e54762ace3753673a98644
🧪 View experiment at: https://dagshub.com/kunalkaushik1982/amazon-product-reviews.mlflow/#/experiments/3


OSError: [WinError 1450] Insufficient system resources exist to complete the requested service